In [2]:
import geatpy as ea
import numpy as np

In [2]:
import linear_model as model
import src.project1.submit.AI as ai
from experiment.评估函数.simulator import Simulator
from src.project1.experimental.贪心不败机 import AI as 贪心不败机
import 算子 as 算子
chessboard_size = 8
time_out = 10

def 组合策略1(Vars):
    PVT, PVT_max = 算子.get_PVT_and_max(Vars[0:10])
    def _组合策略1(chessboard, color, rounds):
        v_position = 算子.value_of_positions(chessboard, color, PVT, PVT_max)
        v_mobility = 算子.value_of_mobility(chessboard, color,  PVT, PVT_max)
        v_edge_sta = 算子.value_of_edge_stability(chessboard, color)
        if rounds<=24:
            return v_position*Vars[10]+v_mobility*Vars[11]+v_edge_sta*Vars[12]
        else:
            return v_position*Vars[13]+v_mobility*Vars[14]+v_edge_sta*Vars[15]
    return _组合策略1

def evalWinRate(Vars, diff=0.01, at_least_times=20):
    wins = 0
    cnts = 0
    prev_prob = np.ones(at_least_times)*100  # 不合法
    prob = -1 # 不合法
    while np.absolute(prob-prev_prob).sum()>at_least_times*diff or prob==1:
        for i_color in [ai.COLOR_BLACK, ai.COLOR_WHITE]:
            agents = {i_color: 贪心不败机(chessboard_size, i_color, time_out, 组合策略1(Vars)),
                  -i_color: 贪心不败机(chessboard_size, -i_color, time_out, 算子.random_base_line)}
            simulator = Simulator(chessboard_size, time_out, agents)
            winner = simulator.quick_run(no_print=True)
            if winner != i_color:
                # print("loss")
                pass
            else:
                # print("win")
                wins += 1
            cnts += 1
            prev_prob = np.append(prev_prob[1:], prob)
            prob = wins/cnts
    print(cnts)
    return prob
# evalWinRate(np.array([9, 10, 18, 5, 19,15, 18, 13, 8, 3, 0.03, 0.4, 0.6, 0.03, 0.6, 0.4]))


In [8]:
# def evalBetterThanNeighbour(Vars):
#     cnt = 0
#     for neighbour in model.neighbours(Vars):
#         for i_color in [ai.COLOR_BLACK, ai.COLOR_WHITE]:
#             agents = {i_color: GreedyAI(chessboard_size, i_color, greedy_fun(Vars)),
#                   -i_color: GreedyAI(chessboard_size, -i_color, greedy_fun(neighbour))}
#             simulator = Simulator(chessboard_size, time_out, agents)
#             winner = simulator.quick_run(no_print=True)
#             if winner != i_color:
#                 cnt+=1
#     # 一般有40次对决
#     return (cnt/40)-0.5 # 违反约束的程度，被很多邻居打败就违反了
# # evalBetterThanNeighbour(np.array([1, 8, 3, 7, 3, 2, 5, 6, 6, 4]))
# # evalBetterThanNeighbour(np.array([9, 10, 18, 5, 19,15, 18, 13, 8, 3]))
# # evalBetterThanNeighbour(np.array([3, 10, 18, 5, 19,15, 18, 13, 8, 3]))
# evalBetterThanNeighbour(np.array([16,14,18,18,20,20,13,12,18,15]))

-0.5

In [1]:
# 构建问题
@ea.Problem.single
def evalVars(Vars):  # 定义目标函数（含约束）
    f = evalWinRate(Vars)  # 计算目标函数值
    # CV = np.array(evalBetterThanNeighbour(Vars))
    CV = np.array([0])
    # f = evalBetterThanNeighbour(Vars)  # 计算目标函数值
    # CV = np.array(0.5-evalWinRate(Vars))
    return f, CV
dim = 16
# varTypes = np.ones(dim) # 整数
varTypes = np.zeros(dim) # 实数
lb = np.ones(dim)*-20
ub = np.ones(dim)*20
lb[10:] = 0
ub[10:] = 20
problem = ea.Problem(name='演化计算求解最优评估函数',
                        M=1,  # 目标维数
                        maxormins=[-1],  # 目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标
                        # maxormins=[1],  # 目标最小最大化标记列表，1：最小化该目标；-1：最大化该目标
                        Dim=dim,  # 决策变量维数
                        varTypes=varTypes,  # 决策变量的类型列表，0：实数；1：整数
                        lb=lb,  # 决策变量下界
                        ub=ub,  # 决策变量上界
                        evalVars=evalVars)
# 构建算法
algorithm = ea.soea_SEGA_templet(problem,
                                    ea.Population(Encoding='RI', NIND=16),
                                    MAXGEN=16,  # 最大进化代数。
                                    # MAXGEN=10,  # 最大进化代数。
                                    logTras=1,  # 表示每隔多少代记录一次日志信息，0表示不记录。
                                    trappedValue=1e-6,  # 单目标优化陷入停滞的判断阈值。
                                    maxTrappedCount=10)  # 进化停滞计数器最大上限值。
# 求解
res = ea.optimize(algorithm, prophet = np.array([[9, 10, 18, 5, 19,15, 18, 13, 8, 3, 0.5, 0.2, 0.3, 0.3, 0.5, 0.2],
                                                 [9, 10, 18, 5, 19,15, 18, 13, 8, 3, 0.03, 0.4, 0.6, 0.03, 0.6, 0.4]]), seed=1, verbose=True, drawing=1, outputMsg=True, drawLog=True, saveFlag=True, dirName='result')

NameError: name 'ea' is not defined